## Set up

In [ ]:
import sys
sys.path.append('./scripts/')
from datasets import *
import scipy.ndimage
import torch
import torchvision 
import os
import numpy as np
from PIL import Image
from torchvision.models import feature_extraction 


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
import get_embeddings
import imp
imp.reload(get_embeddings)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
dataset='ilsvrc12'
layer='Mixed_7b.cat_2'
compute_embeddings_fresh = True
paths, count, y, idx_to_labels = get_dataset(dataset)

print(count, len(paths))

In [ ]:
SAVEFOLD0=f'../outputs/{dataset}'
if not os.path.exists(SAVEFOLD0):
    os.mkdir(SAVEFOLD0)

In [ ]:
try:
    os.mkdir(f"{SAVEFOLD0}/{layer}/")
except:
    print("Maybe the directory already exists? ")
SAVEFOLD=f"{SAVEFOLD0}/{layer}/"

## Get data

In [ ]:
idx_to_labels

In [ ]:
classes = np.unique(y)

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
model.eval()

In [ ]:
 torchvision.models.feature_extraction.get_graph_node_names(model)

In [ ]:
print(layer)
return_nodes={f'{layer}': 'conv', 'avgpool':'avgpool', 'fc':'fc'
             }
model = torchvision.models.feature_extraction.create_feature_extractor(model, return_nodes=return_nodes)

In [ ]:
model

In [ ]:
inputs=torch.zeros((8,3,299,299), dtype=torch.float).to(device)
model.to(device)
outs=model(inputs)
dim_c = outs['conv'].shape[1]
dim_w = outs['conv'].shape[2]
dim_h = outs['conv'].shape[3] 

In [ ]:
if compute_embeddings_fresh == True:
    get_embeddings.get_model_state(model, paths, y, dim_c, dim_w, dim_h, SAVEFOLD=SAVEFOLD)

In [ ]:
print("\ndevice: ")
print(device )
print("\ntorch.cuda.is_available(): ")
print(torch.cuda.is_available())
print("\ntorch.cuda.device_count()")
print(torch.cuda.device_count())
print("\ntorch.cuda.get_device_name(0)")
print(torch.cuda.get_device_name(0))
print("\ninputs.is_cuda: ")
print(inputs.is_cuda)

In [ ]:
print(torch.cuda.get_device_name(1))

## Load precomputed results

In [ ]:
gradients_wrt_conv_layer=np.load(f"{SAVEFOLD}/gradients_wrt_conv_layer.npy")
predictions=np.load(f"{SAVEFOLD}/predictions.npy")
conv_maps=np.load(f"{SAVEFOLD}/conv_maps.npy")